In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import os
from pathlib import Path

# set cwd to src
os.chdir('../src')
from utils import *

data_path = Path('../data')

In [2]:
trips_df = pd.read_csv(data_path / 'dest_trips_IrvineSpectrumCenter_2023-01-01_to_2023-01-31.csv')
parking_gpd = gpd.read_file(data_path / 'parking_Irvine_2023-01-01_2023-01-31.csv').set_crs("EPSG:4326")

AttributeError: 'DataFrame' object has no attribute 'set_crs'

In [ ]:
display(trips_df)
display(parking_gpd)

,year,month,day,hour,minute,second,orig_qk17,dest_qk17,start_lat,start_lon,end_lat,end_lon,count,trip_id,is_moving
0,2023,1,11,1,49,43,2301320201100220,2301320210201011,33.708891,-117.858685,33.649363,-117.740551,1,659e88b86beff6c222eb12794415d0b3,1
1,2023,1,30,18,59,33,2301320201310232,2301320210201033,33.633631,-117.810590,33.643506,-117.742996,1,f484952f7990acbe3870127dc03d23e9,1
2,2023,1,12,20,23,38,2301320021302221,2301320210201032,33.910793,-117.858157,33.642677,-117.745207,1,c8d85398153f165ec66abe7b95209c92,1
3,2023,1,2,21,25,16,2301320212332122,2301320210201022,33.440676,-117.629412,33.643763,-117.750153,1,20c4adc62298ba73c80f7b646de27612,1
4,2023,1,2,1,33,24,2301320212332322,2301320210023232,33.431988,-117.628030,33.652833,-117.744219,1,85bac8b17a650401a1a2221a4c002b24,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44771,2023,1,19,0,44,19,2301320201322333,2301320210201013,33.578710,-117.841021,33.647776,-117.742490,1,53040539d22eebf4a7134855749792af,1
44772,2023,1,12,22,56,28,2301231130122203,2301320210023221,33.949078,-118.385690,33.655667,-117.747356,1,eda331497c4563d4c7b4a7a91db150f3,1
44773,2023,1,19,23,29,52,2301320210201030,2301320210201013,33.645700,-117.744880,33.647629,-117.742296,1,a1407768799141c8667ccb463276d4ad,1
44774,2023,1,29,20,54,3,2301320210210301,2301320210201031,33.640745,-117.715283,33.644925,-117.742459,1,e88d462149156ab366e5b5afa5dacf89,1


,dt_start_date,dt_end_date,i_avail,i_occ,f_pct_occ,pk_lot_alias,f_occupancy_rank,pk_lot,location,geometry
0,2023-01-31 07:41:55.166454+00:00,2023-01-31 11:41:55.166454+00:00,184,81.0,31.0,4449d7eb-815a-4293-ab6a-2add6a19fc3c,3.0,90394,POINT(-117.859954615167 33.6817585441798),"MULTIPOLYGON(((-117.86031 33.68066,-117.86083 ..."
1,2023-01-31 07:41:55.137835+00:00,2023-01-31 11:41:55.137835+00:00,174,248.0,59.0,35012bdd-88f0-470d-a710-6513579ac35e,3.0,90394,POINT(-117.859954615167 33.6817585441798),"MULTIPOLYGON(((-117.86031 33.68066,-117.86083 ..."
2,2023-01-31 07:41:55.109223+00:00,2023-01-31 11:41:55.109223+00:00,43,294.0,87.0,10a84b43-9a9c-4b78-8539-a48121f1eae9,3.0,90394,POINT(-117.859954615167 33.6817585441798),"MULTIPOLYGON(((-117.86031 33.68066,-117.86083 ..."
3,2023-01-31 07:41:55.079620+00:00,2023-01-31 11:41:55.079620+00:00,254,193.0,43.0,6e504244-38c2-4657-ac01-cd1fb94e497d,3.0,90394,POINT(-117.859954615167 33.6817585441798),"MULTIPOLYGON(((-117.86031 33.68066,-117.86083 ..."
4,2023-01-31 06:56:32.707441+00:00,2023-01-31 10:56:32.707441+00:00,178,87.0,33.0,4449d7eb-815a-4293-ab6a-2add6a19fc3c,3.0,90394,POINT(-117.859954615167 33.6817585441798),"MULTIPOLYGON(((-117.86031 33.68066,-117.86083 ..."
...,...,...,...,...,...,...,...,...,...,...
27169,2023-01-02 01:12:39.448474+00:00,2023-01-02 05:12:39.448474+00:00,44,,80.0,2720e306-4fc9-4abe-8ccb-78239a14c004,3.0,375750,POINT(-117.7398482 33.6484276),"MULTIPOLYGON(((-117.74027 33.64827,-117.74028 ..."
27170,2023-01-02 01:00:08.721957+00:00,2023-01-02 05:00:08.721957+00:00,46,,79.0,2720e306-4fc9-4abe-8ccb-78239a14c004,3.0,375750,POINT(-117.7398482 33.6484276),"MULTIPOLYGON(((-117.74027 33.64827,-117.74028 ..."
27171,2023-01-01 23:55:37.398795+00:00,2023-01-02 03:55:37.398795+00:00,45,,80.0,2720e306-4fc9-4abe-8ccb-78239a14c004,3.0,375750,POINT(-117.7398482 33.6484276),"MULTIPOLYGON(((-117.74027 33.64827,-117.74028 ..."
27172,2023-01-01 23:47:09.975869+00:00,2023-01-02 03:47:09.975869+00:00,45,,80.0,2720e306-4fc9-4abe-8ccb-78239a14c004,3.0,375750,POINT(-117.7398482 33.6484276),"MULTIPOLYGON(((-117.74027 33.64827,-117.74028 ..."


In [ ]:
print(type(parking_gpd))

<class 'pandas.core.frame.DataFrame'>
